In [1]:
from pyevolve import G1DList
from pyevolve import GSimpleGA
from pyevolve import Crossovers
from pyevolve import Consts

import random
from math import sqrt

In [2]:
PIL_SUPPORT = None

try:
    from PIL import Image, ImageDraw, ImageFont
    PIL_SUPPORT = True
except:
    PIL_SUPPORT = False

In [4]:
cm     = []
coords = []
CITIES = 30
WIDTH   = 1024
HEIGHT  = 768
LAST_SCORE = -1

In [5]:
def cartesian_matrix(coords):
    """ A distance matrix """
    matrix={}
    for i,(x1,y1) in enumerate(coords):
        for j,(x2,y2) in enumerate(coords):
            dx, dy = x1-x2, y1-y2
            dist=sqrt(dx*dx + dy*dy)
            matrix[i,j] = dist
    return matrix

In [6]:
def tour_length(matrix, tour):
    """ Returns the total length of the tour """
    total = 0
    t = tour.getInternalList()
    for i in range(CITIES):
        j      = (i+1)%CITIES
        total += matrix[t[i], t[j]]
    return total

In [7]:
def write_tour_to_img(coords, tour, img_file):
    """ The function to plot the graph """
    padding=20
    coords=[(x+padding,y+padding) for (x,y) in coords]
    maxx,maxy=0,0
    for x,y in coords:
        maxx, maxy = max(x,maxx), max(y,maxy)
    maxx+=padding
    maxy+=padding
    img=Image.new("RGB",(int(maxx),int(maxy)),color=(255,255,255))
    font=ImageFont.load_default()
    d=ImageDraw.Draw(img);
    num_cities=len(tour)
    for i in range(num_cities):
        j=(i+1)%num_cities
        city_i=tour[i]
        city_j=tour[j]
        x1,y1=coords[city_i]
        x2,y2=coords[city_j]
        d.line((int(x1),int(y1),int(x2),int(y2)),fill=(0,0,0))
        d.text((int(x1)+7,int(y1)-5),str(i),font=font,fill=(32,32,32))

    for x,y in coords:
        x,y=int(x),int(y)
        d.ellipse((x-5,y-5,x+5,y+5),outline=(0,0,0),fill=(196,196,196))
    del d
    img.save(img_file, "PNG")
    print "The plot was saved into the %s file." % (img_file,)

In [8]:
def G1DListTSPInitializator(genome, **args):
    """ The initializator for the TSP """
    lst = [i for i in xrange(genome.getListSize())]
    random.shuffle(lst)
    genome.setInternalList(lst)

In [9]:
def evolve_callback(ga_engine):
    global LAST_SCORE
    if ga_engine.getCurrentGeneration() % 100 == 0:
        best = ga_engine.bestIndividual()
        if LAST_SCORE != best.getRawScore():
            write_tour_to_img( coords, best, "tspimg/tsp_result_%d.png" % ga_engine.getCurrentGeneration())
            LAST_SCORE = best.getRawScore()
    return False

In [11]:
global cm, coords, WIDTH, HEIGHT

coords = [(random.randint(0, WIDTH), random.randint(0, HEIGHT))
             for i in xrange(CITIES)]
cm     = cartesian_matrix(coords)
genome = G1DList.G1DList(len(coords))

genome.evaluator.set(lambda chromosome: tour_length(cm, chromosome))
genome.crossover.set(Crossovers.G1DListCrossoverEdge)
genome.initializator.set(G1DListTSPInitializator)

ga = GSimpleGA.GSimpleGA(genome)
ga.setGenerations(10000)
ga.setMinimax(Consts.minimaxType["minimize"])
ga.setCrossoverRate(1.0)
ga.setMutationRate(0.02)
ga.setPopulationSize(80)

ga.stepCallback.set(evolve_callback)

In [12]:
ga.evolve(freq_stats=200)
best = ga.bestIndividual()

if PIL_SUPPORT:
    write_tour_to_img(coords, best, "tsp_result.png")
else:
    print "No PIL detected, cannot plot the graph !"

The plot was saved into the tspimg/tsp_result_0.png file.
Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw) [16159.36(16460.77)/11123.67(10861.51)/13466.14(13466.14)]
The plot was saved into the tspimg/tsp_result_100.png file.
The plot was saved into the tspimg/tsp_result_200.png file.
Gen. 200 (2.00%): Max/Min/Avg Fitness(Raw) [8408.07(9221.75)/5361.85(4406.78)/7006.73(7006.73)]
The plot was saved into the tspimg/tsp_result_400.png file.
Gen. 400 (4.00%): Max/Min/Avg Fitness(Raw) [7827.30(9675.01)/5630.35(4366.38)/6522.75(6522.75)]
The plot was saved into the tspimg/tsp_result_500.png file.
The plot was saved into the tspimg/tsp_result_600.png file.
Gen. 600 (6.00%): Max/Min/Avg Fitness(Raw) [7068.36(9448.82)/5257.91(3980.06)/5890.30(5890.30)]
The plot was saved into the tspimg/tsp_result_700.png file.
The plot was saved into the tspimg/tsp_result_800.png file.
Gen. 800 (8.00%): Max/Min/Avg Fitness(Raw) [7074.03(9064.88)/5171.60(3950.03)/5895.02(5895.02)]
The plot was saved into the tspimg/tsp